## POS tagging using modified Viterbi

### Data Preparation

In [222]:

#Importing libraries
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

In [223]:
# Get the sentences of sample test file.
test_sentences = [];

myfile = open("Test_sentences.txt")
for line in myfile:
   test_sentences.append(line)
myfile.close()
test_sentences



['Android is a mobile operating system developed by Google.\n',
 'Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.\n',
 "Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.\n",
 'Twitter is an online news and social networking service on which users post and interact with messages known as tweets.\n',
 'Before entering politics, Donald Trump was a domineering businessman and a television personality.\n',
 'The 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.\n',
 'This is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe.\n',
 'Show me the cheapest round trips from Dallas to Atlanta\n',
 'I would like to see flights from Denver to Philadelphia.\n',
 'Show me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco.\n',
 'NASA invited social medi

In [224]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [225]:
# first few tagged sentences
print(nltk_data[:40])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], [('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'), ('

In [226]:
# Splitting into train and test
random.seed(1234)
train_set, test_set = train_test_split(nltk_data,test_size=0.5)

print(len(train_set))
print(len(test_set))
print(train_set[:40])

1957
1957
[[('But', 'CONJ'), ('many', 'ADJ'), ('banks', 'NOUN'), ('are', 'VERB'), ('turning', 'VERB'), ('away', 'ADV'), ('from', 'ADP'), ('strict', 'ADJ'), ('price', 'NOUN'), ('competition', 'NOUN'), ('.', '.')], [('He', 'PRON'), ('said', 'VERB'), ('0', 'X'), ('Equitable', 'NOUN'), ('hopes', 'VERB'), ('*-1', 'X'), ('to', 'PRT'), ('eventually', 'ADV'), ('reduce', 'VERB'), ('its', 'PRON'), ('stake', 'NOUN'), ('in', 'ADP'), ('Younkers', 'NOUN'), ('to', 'PRT'), ('less', 'ADJ'), ('than', 'ADP'), ('50', 'NUM'), ('%', 'NOUN'), ('.', '.')], [('That', 'DET'), ('can', 'VERB'), ('pay', 'VERB'), ('off', 'PRT'), ('down', 'ADP'), ('the', 'DET'), ('road', 'NOUN'), ('as', 'ADP'), ('customers', 'NOUN'), (',', '.'), ('especially', 'ADV'), ('the', 'DET'), ('younger', 'ADJ'), ('ones', 'NOUN'), (',', '.'), ('change', 'VERB'), ('from', 'ADP'), ('borrowers', 'NOUN'), ('to', 'PRT'), ('savers\\/investors', 'NOUN'), ('.', '.')], [('Mr.', 'NOUN'), ('Coleman', 'NOUN'), ('said', 'VERB'), ('this', 'DET'), ('week', 

In [227]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

50317

In [228]:
# tokens 
tokens = [pair[0] for pair in train_tagged_words]
tokens[:10]

['But',
 'many',
 'banks',
 'are',
 'turning',
 'away',
 'from',
 'strict',
 'price',
 'competition']

In [229]:
# vocabulary
V = set(tokens)
print(len(V))

8539


In [230]:
# number of tags
T = set([pair[1] for pair in train_tagged_words])
len(T)
T

{'.',
 'ADJ',
 'ADP',
 'ADV',
 'CONJ',
 'DET',
 'NOUN',
 'NUM',
 'PRON',
 'PRT',
 'VERB',
 'X'}

In [231]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [232]:
# creating t x t transition matrix of tags

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [233]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))

### Build the vanilla Viterbi based POS tagger

In [234]:
# Viterbi Algorithm.
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [235]:
# Running on entire test dataset would take more than 3-4hrs. 
# Let's test our Viterbi algorithm on a few sample sentences of test dataset

random.seed(1234)

# choose random 5 sents
rndom = [random.randint(1,len(test_set)) for x in range(5)]

# list of sents
test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
test_tagged_words

['``',
 'They',
 'view',
 'this',
 'as',
 'a',
 'growth',
 'area',
 'so',
 'they',
 'went',
 'about',
 'it',
 'with',
 'a',
 'systematic',
 'approach',
 ',',
 "''",
 'says',
 '*T*-1',
 'Richard',
 'Olsen',
 ',',
 'a',
 'Candela',
 'vice',
 'president',
 '.',
 'American',
 'Express',
 'Co.',
 'and',
 'General',
 'Motors',
 'Corp.',
 "'s",
 'beleaguered',
 'Buick',
 'division',
 'are',
 'joining',
 'forces',
 'in',
 'a',
 'promotion',
 'aimed',
 '*',
 'at',
 '*',
 'boosting',
 'Buick',
 "'s",
 'sales',
 'while',
 '*',
 'encouraging',
 'broader',
 'use',
 'of',
 'the',
 'American',
 'Express',
 'card',
 '.',
 '``',
 'It',
 'has',
 '*-1',
 'to',
 'weigh',
 ',',
 'on',
 'the',
 'other',
 'hand',
 ',',
 'the',
 'relatively',
 'high',
 'price',
 'of',
 'sugar',
 '0',
 'it',
 'can',
 'earn',
 '*T*-3',
 'on',
 'the',
 'export',
 'market',
 'in',
 '*',
 'making',
 'decisions',
 'as',
 'to',
 'whether',
 '*',
 'to',
 'produce',
 'sugar',
 'or',
 'alcohol',
 ',',
 "''",
 'Mr.',
 'Oxnard',
 'said',

In [ ]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)
print(tagged_seq)

In [ ]:
## Testing
sentence_test = 'Android is a mobile operating system developed by Google. Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013. Google and Twitter made a deal in 2015 that gave Google access to Twitter\'s firehose. Twitter is an online news and social networking service on which users post and interact with messages known as tweets. Before entering politics, Donald Trump was a domineering businessman and a television personality.'
words = word_tokenize(sentence_test)

start = time.time()
test_tagged_seq = Viterbi(words)
end = time.time()
difference = end-start


print("Time taken in seconds: ", difference)
print(test_tagged_seq)




### Solve the problem of unknown words

In [ ]:
# Lexicon Tagger.
unigram_tagger = nltk.UnigramTagger(train_set)

In [ ]:
# specify patterns for tagging
patterns = [
    (r'.*ing$', 'VBG'),              # gerund
    (r'.*ed$', 'VBD'),               # past tense
    (r'.*es$', 'VBZ'),               # 3rd singular present
    (r'.*ould$', 'MD'),              # modals
    (r'.*\'s$', 'NN$'),              # possessive nouns
    (r'.*s$', 'NNS'),                # plural nouns
    (r'.*', 'NN')                    # nouns
]

In [ ]:
regexp_tagger = nltk.RegexpTagger(patterns)

#### Evaluating tagging accuracy

In [ ]:
unigram_tagger.evaluate(test_set)

In [ ]:
# Rule-Based Tagger
regexp_tagger = nltk.RegexpTagger(patterns)
regexp_tagger.evaluate(test_set)

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

In [ ]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 

In [ ]:
accuracy = len(check)/len(tagged_seq)

In [ ]:
accuracy

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

In [ ]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)